## This will be a space where I play out any further analyses

In [1]:
import pandas as pd

In [2]:

presdebate = pd.read_csv("/Users/emmatarcson/Documents/data_science/RhetoricalFactor-analysis/data/only_D_and_R/welkerPRESdebate.csv")
presdebate = presdebate[['Aff','transcript']]
presdebate.Aff.value_counts()

R    193
D    131
Name: Aff, dtype: int64

In [3]:
import spacy

In [32]:
from spacy.symbols import ADJ
import en_core_web_sm
nlp = en_core_web_sm.load()
adjs = []
adj = [adjs.append(t.text) for tok in presdebate['transcript'] for t in nlp(tok) if t.pos==ADJ]

In [35]:
import pandas as pd
from textblob import TextBlob

ModuleNotFoundError: No module named 'textblob'